# 1.0 Importing Libraries

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from matplotlib import pyplot as plt

In [2]:
pd.set_option('display.max_columns', 50)

# 2.0 Loading Data

In [3]:
df_sales_group = pd.read_csv('dataset/sales_group.csv')

author = pd.read_csv('dataset/author.csv')
award = pd.read_csv('dataset/award.csv')
book = pd.read_csv('dataset/book.csv')
edition = pd.read_csv('dataset/edition.csv')
format = pd.read_csv('dataset/format.csv')
genders = pd.read_csv('dataset/genders.csv')
info = pd.read_csv('dataset/info.csv')
publisher = pd.read_csv('dataset/publisher.csv')
ratings = pd.read_csv('dataset/ratings.csv')
series = pd.read_csv('dataset/series.csv')

### 2.1 Basic Data Transformation

In [4]:
# Renaming the columns on the award dataset to eliminate spaces
cols_new = ['book_id', 'title', 'award_name', 'year_won']
award.columns = cols_new

In [5]:
# Checking for duplicates
datasets = [author, award, book, edition, format, info, publisher, series, genders, ratings]
datasets_names = ['author', 'award', 'book', 'edition', 'format', 'info', 'publisher', 'series', 'genders', 'ratings']

i=0
for dataset in datasets:
    print(f'Dataset: {datasets_names[i]} - Quantidade de duplicadas: {dataset.duplicated().sum()}\n')
    i+=1

Dataset: author - Quantidade de duplicadas: 0

Dataset: award - Quantidade de duplicadas: 0

Dataset: book - Quantidade de duplicadas: 0

Dataset: edition - Quantidade de duplicadas: 0

Dataset: format - Quantidade de duplicadas: 0

Dataset: info - Quantidade de duplicadas: 0

Dataset: publisher - Quantidade de duplicadas: 119

Dataset: series - Quantidade de duplicadas: 0

Dataset: genders - Quantidade de duplicadas: 0

Dataset: ratings - Quantidade de duplicadas: 0



In [6]:
# Lets remove the duplicates from the publisher dataset
publisher.drop_duplicates(inplace=True)

### 2.2 Merging the datasets

In [7]:
df = pd.merge(book, author, how='left', on='author_id')
df = pd.merge(df, info, how='left', on='book_id')
df = pd.merge(df, series, how='left', on='series_id')
df = pd.merge(df, genders, how='left', on='genre_id')

#### 2.2.1 Award dataset

The award dataset has duplicated book_ids for books with multiple awards, lets aggregate them before merging

In [8]:
award_agg = award.groupby('book_id').agg({
    'title': 'first',
    'award_name': lambda x: ', '.join(x),
    'year_won': 'first'}).reset_index()

df = pd.merge(df, award_agg, how='left', on='book_id')

#### 2.2.2 Publisher dataset

The publisher dataset also has duplicated book_ids for books with multiple publishers. But, we have a problem that some publishers have two different pub_ids, so we can't merge them yet.

So, first we'll determine that the right pub_id is the one who corresponds to the initials of the publisher name.


In [9]:
# Defining useful functions

def get_initials(name):
    return "".join(word[0] for word in name.split()).upper()

def select_pub_id(group):
    initials = group["initials"].iloc[0]
    matching_pub_id = group[group["pub_id"] == initials]

    if not matching_pub_id.empty:
        return matching_pub_id.iloc[0]
    else:
        return group.sort_values("pub_id").iloc[0]


In [10]:
# Getting the initials of the publisher name
publisher["initials"] = publisher["name"].apply(get_initials)

# Selecting the correct pub_id
publisher_corrected = publisher.groupby(["book_id", "name"], group_keys=False).apply(select_pub_id)
publisher_corrected.reset_index(drop=True, inplace=True)

# Dropping the initial column
publisher_corrected = publisher_corrected.drop(columns=["initials"])
publisher = publisher.drop(columns=["initials"])

Now we can aggregate and merge the datasets.

Here, we'll opt for summing the marketing spend, since it's the corresponding value for each book

In [11]:
# Aggregating the publisher dataset
publisher_agg = publisher_corrected.groupby('book_id').agg({
    'name': lambda x: ', '.join(x),
    'city': lambda x: ', '.join(x),
    'state': 'first',
    'country': 'first',
    'year_established': 'first',
    'marketing_spend': 'sum',
    'pub_id': lambda x: ', '.join(x)
}).reset_index()

In [12]:
# Merging the datasets
df = pd.merge(df, publisher_agg, how='left', on='book_id')

#### 2.2.3 Ratings dataset

Let's keep only the average rating and the number of ratings for each book

In [13]:
# Getting the average rating
ratings_mean = ratings.groupby('book_id')['rating'].mean().round(2).reset_index()
ratings_mean.columns = ['book_id', 'rating_mean']

In [14]:
# Getting the number of ratings
ratings_count = ratings.groupby('book_id')['rating'].count().reset_index()
ratings_count.columns = ['book_id', 'rating_count']

In [15]:
# Merging the datasets
df = pd.merge(df, ratings_mean, how='left', on='book_id')
df = pd.merge(df, ratings_count, how='left', on='book_id')

#### 2.2.4 Edition dataset

Lets first merge the edition and the format datasets.

In [16]:
df_edition = pd.merge(edition, format, how='left', on='format_id')

In [17]:
df_edition.drop(columns=['format_id'], inplace=True)

#### 2.2.5 Sales dataset

In [18]:
sales = pd.merge(df_sales_group, df_edition, how='left', on='isbn')
# sales = pd.merge(sales, df, how='left', on=['book_id', 'pub_id'])

# 3.0 Data Transformation

Lets organize the data before merging into the sales dataset

In [19]:
# Getting the author name
df['author_name'] = df['first_name'] + ' ' + df['last_name']

## 3.1 Dataset columns

In [20]:
# Remove unuseful columns
df.drop(columns=['title_y', 'author_id', 'genre_id', 'series_id', 'first_name', 'last_name'], inplace=True)

In [21]:
# Renaming the columns
df.rename(columns={
    'title_x': 'title',
    'genre_desc': 'genre',
    'name': 'pub_name',
    'city': 'pub_city',
    'state': 'pub_state',
    'country': 'pub_country'
}, inplace=True)

In [24]:
sales.rename(columns={
    'COUNT': 'sales_count',
    'format_desc': 'format'
}, inplace=True)

## 3.2 Data types

In [44]:
# Checking the data types
df.dtypes

book_id                      object
title                        object
birthday             datetime64[ns]
country_residence            object
hrs_writing_day             float64
volume_number                 Int64
series_name                  object
genre                        object
award_name                   object
year_won                      Int64
pub_name                     object
pub_city                     object
pub_state                    object
pub_country                  object
year_established              Int64
marketing_spend               Int64
pub_id                       object
rating_mean                 float64
rating_count                  Int64
author_name                  object
dtype: object

In [45]:
sales.dtypes

sale_date           datetime64[ns]
isbn                        object
sales_count                  int64
book_id                     object
pub_id                      object
publication_date    datetime64[ns]
pages                        Int64
print_run_size_k             Int64
price                      float64
format                      object
dtype: object

In [38]:
# Transform from object to datetime
df['birthday'] = pd.to_datetime(df['birthday'], format='%d/%m/%Y')
sales['sale_date'] = pd.to_datetime(sales['sale_date'], format='%d/%m/%Y')
sales['publication_date'] = pd.to_datetime(sales['publication_date'], format='%d/%m/%Y')

In [ ]:
# Transform from float to int
df['volume_number'] = df['volume_number'].astype('Int64')
df['year_won'] = df['year_won'].astype('Int64')
df['year_established'] = df['year_established'].astype('Int64')
df['marketing_spend'] = df['marketing_spend'].astype('Int64')
df['rating_count'] = df['rating_count'].astype('Int64')
sales['print_run_size_k'] = sales['print_run_size_k'].astype('Int64')
sales['pages'] = sales['pages'].astype('Int64')

## 3.3 Null values

In [52]:
# Checking for missing values
df.isnull().sum()

book_id               0
title                 0
birthday              0
country_residence     0
hrs_writing_day       0
volume_number         0
series_name           0
genre                 0
award_name           41
year_won             41
pub_name              1
pub_city              1
pub_state             1
pub_country           1
year_established      1
marketing_spend       1
pub_id                1
rating_mean           2
rating_count          0
author_name           0
dtype: int64

In [48]:
sales.isnull().sum()

sale_date           0
isbn                0
sales_count         0
book_id             0
pub_id              0
publication_date    0
pages               0
print_run_size_k    0
price               0
format              0
dtype: int64

In [51]:
# volume_number null values are the ones that don't have series
df['volume_number'].fillna(0, inplace=True)

# series_name null values are the ones that also don't have series
df['series_name'].fillna('', inplace=True)

# award_name and year_won null values are the ones that don't have awards, so let's keep them as NaN

# rating_count null values are the ones that don't have ratings
df['rating_count'].fillna(0, inplace=True)

# We have one book that doesn't have a publisher
# Let's keep the rating_mean null value as NaN to not affect the average rating


## 3.4 Check duplicates

In [53]:
df.duplicated().sum()

0

In [54]:
df['book_id'].duplicated().sum()

0

In [55]:
sales.duplicated().sum()

0

## Final merge

In [61]:
print(f'The df dataset has information about {df.shape[0]} books and the following {df.shape[1]} columns:\n{list(i for i in df.columns)}')

The df dataset has information about 58 books and the following 20 columns:
['book_id', 'title', 'birthday', 'country_residence', 'hrs_writing_day', 'volume_number', 'series_name', 'genre', 'award_name', 'year_won', 'pub_name', 'pub_city', 'pub_state', 'pub_country', 'year_established', 'marketing_spend', 'pub_id', 'rating_mean', 'rating_count', 'author_name']


In [62]:
print(f'The sales dataset has information about {sales.shape[0]} sales and the following {sales.shape[1]} columns:\n{list(i for i in sales.columns)}')

The sales dataset has information about 15188 sales and the following 10 columns:
['sale_date', 'isbn', 'sales_count', 'book_id', 'pub_id', 'publication_date', 'pages', 'print_run_size_k', 'price', 'format']


Now we can finally merge the two datasets.

In [63]:
df_sales = pd.merge(sales, df, how='left', on=['book_id', 'pub_id'])

# Tests

In [69]:
df_sales.head()

,sale_date,isbn,sales_count,book_id,pub_id,publication_date,pages,print_run_size_k,price,format,title,birthday,country_residence,hrs_writing_day,volume_number,series_name,genre,award_name,year_won,pub_name,pub_city,pub_state,pub_country,year_established,marketing_spend,rating_mean,rating_count,author_name
0,2193-01-02,989-28-3705-007-2,5,HP265,CHP,2188-06-03,16,55,5.99,Board book,Heliotrope Pajamas,2141-01-31,Hong Kong,6.00,0,,Childrens,Newberry Medal,2182,Cedar House Publishers,Friday Harbor,Washington,USA,1906,72000,4.54,1764,Malin Wolff
1,2193-01-02,989-28-79-11297-4,3,TP887,ESP,2192-08-25,1296,30,13.46,Trade paperback,the life and times of an utterly inconsequenti...,2157-02-26,United States,5.13,0,,Fiction,PEN/Faulkner Award,2192,Etaoin Shrdlu Press,Seattle,Washington,USA,1889,2320000,3.20,1369,David Beam
2,2193-01-02,989-28-79-18127-7,1,AY135,ESP,2179-04-24,704,15,27.99,Hardcover,And I Said Yes,2129-07-11,Norway,6.00,0,,Fiction,NaN,<NA>,Etaoin Shrdlu Press,Seattle,Washington,USA,1889,2320000,3.86,970,Elmer Komroff
3,2193-01-02,989-28-79-82197-5,2,TC188,ESP,2186-12-05,469,20,12.50,Trade paperback,Thatchwork Cottage,2145-01-18,Brazil,6.00,0,,Fiction,NaN,<NA>,Etaoin Shrdlu Press,Seattle,Washington,USA,1889,2320000,4.21,708,Burton Malamud
4,2193-01-02,989-28-79-05638-4,1,TC188,ESP,2185-11-15,485,12,27.99,Hardcover,Thatchwork Cottage,2145-01-18,Brazil,6.00,0,,Fiction,NaN,<NA>,Etaoin Shrdlu Press,Seattle,Washington,USA,1889,2320000,4.21,708,Burton Malamud
